In [ ]:
import pandas as pd

df = pd.read_csv('data/data4.tsv', delimiter = '\t')

df

In [ ]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

imgs = np.array([ img_to_array(load_img(i)) for i in df['image'].values ])

imgs.shape

In [ ]:
cols = ['x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7', 'x8', 'y8']

prob = [1.0, 1.0, 1.0, 1.0, 0.8, 0.8, 0.6, 0.6, 0.6]

def around(n):
    #return [(x, y) for x in range(-n, n + 1) for y in range(-n, n + 1) if abs(x) + abs(y) == n]
    return [(x, y) for x in range(-n, n + 1) for y in range(-n, n + 1) if abs(x) == n or abs(y) == n]

labels = np.zeros(imgs.shape[:-1] + (int(len(cols) / 2) + 1,))
labels[:, :, :, 0] = 1.0

tmp_labels = df[cols].values

for i in range(len(tmp_labels)):
    label = labels[i]
    ts = tmp_labels[i]
    
    for j in range(0, len(ts), 2):
        x = ts[j]
        y = ts[j + 1]
        c = int(j / 2) + 1
        
        for k in range(len(prob)):
            p = prob[k]
            
            for a in around(k):
                ax = x + a[0]
                ay = y + a[1]
                
                if ax >= 0 and ax < imgs.shape[2] and ay >= 0 and ay < imgs.shape[1]:
                    label[ay, ax, :] = 0.0
                    label[ay, ax, c] = p
                    label[ay, ax, 0] = 1.0 - p
        
labels.shape

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

def show_label(index):
    plt.rcParams['figure.figsize'] = (4, 4)
    plt.imshow(labels[index, :, :, 1:].sum(axis = -1), cmap = 'gray')


In [ ]:
show_label(0)

In [ ]:
show_label(1)

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Dropout, UpSampling2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.normalization import BatchNormalization

input = Input(shape = imgs.shape[1:])
x = input

x = BatchNormalization()(x)

x = Conv2D(16, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(16, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = MaxPool2D()(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Conv2D(128, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)

x = UpSampling2D()(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(64, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = UpSampling2D()(x)
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(32, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = UpSampling2D()(x)
x = Conv2D(16, 3, padding = 'same', activation = 'relu')(x)
x = Conv2D(16, 3, padding = 'same', activation = 'relu')(x)

x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(64, activation = 'relu')(x)
x = Dropout(0.3)(x)

output = Dense(labels.shape[-1], activation = 'softmax')(x)

model = Model(inputs = input, outputs = output)

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

model.summary()

In [ ]:
from keras.models import load_model

model = load_model('model/cnn-5c_1_0.h5')

In [ ]:
wg = np.ones(labels.shape[-1]) * (imgs.shape[1] * imgs.shape[2])
wg[0] = 1

print(wg)

hist = model.fit(imgs, labels, initial_epoch = 0, epochs = 100, batch_size = 10, class_weight = wg)
hist

In [ ]:
plt.rcParams['figure.figsize'] = (6, 3)

plt.subplot(1, 2, 1)
plt.plot(hist.history['loss'])

plt.subplot(1, 2, 2)
plt.plot(hist.history['acc'])

In [ ]:
model.save('model/cnn-5c_1_0.h5')

In [ ]:
import cv2

color_set = [(255, 255, 255), (255, 75, 0), (255, 241, 0), (3, 175, 122), (0, 90, 255), 
              (77, 196, 255), (255, 128, 130), (246, 170, 0), (153, 0, 153), (128, 64, 0)]

def predict(index, top_n = 5, circle_r = 3, s = 12.0):
    plt.rcParams['figure.figsize'] = (s, s)

    img = imgs[index]

    p = model.predict(np.array([img]))[0]

    img1 = np.apply_along_axis(lambda x: color_set[x.argmax()], -1, labels[index])
    
    plt.subplot(1, 4, 1)
    plt.imshow(img1)

    img2 = np.apply_along_axis(lambda x: color_set[x.argmax()], -1, p)
    
    plt.subplot(1, 4, 2)
    plt.imshow(img2)

    img3 = cv2.addWeighted(img.astype(int), 0.4, img2, 0.6, 0)

    plt.subplot(1, 4, 3)
    plt.imshow(img3)

    img4 = img.astype(int)

    for i in range(1, labels.shape[-1]):
        print(f"----- category {i} : x={tmp_labels[index, (i - 1) * 2]}, y={tmp_labels[index, (i - 1) * 2 + 1]} -----")
        
        r = p[:, :, i]
        
        for a in np.argsort(r.flatten())[::-1][:top_n]:
            y, x = divmod(a, imgs.shape[2])
            
            print(f"index={a}, x={x}, y={y}, prob={r[y, x]}")
            
            img4 = cv2.circle(img4, (x, y), circle_r, color_set[i], -1)


    plt.subplot(1, 4, 4)
    plt.imshow(img4)

In [ ]:
def predict_eval(file, top_n = 5, circle_r = 3, s = 12.0):
    plt.rcParams['figure.figsize'] = (s, s)

    img = img_to_array(load_img(file))

    p = model.predict(np.array([img]))[0]

    img1 = np.apply_along_axis(lambda x: color_set[x.argmax()], -1, p)
    
    plt.subplot(1, 3, 1)
    plt.imshow(img1)

    img2 = cv2.addWeighted(img.astype(int), 0.4, img1, 0.6, 0)

    plt.subplot(1, 3, 2)
    plt.imshow(img2)

    img3 = img.astype(int)

    for i in range(1, labels.shape[-1]):
        print(f"----- category {i} -----")
        
        r = p[:, :, i]
        
        for a in np.argsort(r.flatten())[::-1][:top_n]:
            y, x = divmod(a, imgs.shape[2])
            
            print(f"index={a}, x={x}, y={y}, prob={r[y, x]}")
            
            img3 = cv2.circle(img3, (x, y), circle_r, color_set[i], -1)


    plt.subplot(1, 3, 3)
    plt.imshow(img3)

In [ ]:
predict(0)

In [ ]:
predict(1)

In [ ]:
predict(2)

In [ ]:
predict(3)

In [ ]:
predict_eval('img2/01.jpg')